## RightMove Scrape

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
#Testing
index = 2

#r1= requests.get('https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=OUTCODE%5E1864&minPrice=550000&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords=')
#r3= requests.get(f'https://www.rightmove.co.uk/property-for-sale/find.html?includeSSTC=false&keywords=&sortType=2&minPrice={minPrice}&viewType=LIST&channel=BUY&index=0&maxPrice={maxPrice}&radius=0.5&locationIdentifier=OUTCODE%5E1864')
#r2= requests.get(f'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=STATION%5E6095&maxPrice={maxPrice}&minPrice={minPrice}&radius={radius}&index=24&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords=')
#r3= requests.get(f'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=STATION%5E6095&maxPrice={maxPrice}&minPrice={minPrice}&radius={radius}&index={index*24}&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords=')

### Main Webscraper

**Loops through all n rightmove pages extracting details of each property.**

1. First we define the parameters of our search i.e. the max/min price and the radius around the station.
2. Next, since the URL changes for page 1 vs pages 2+, we reconfigure the request accordingly using `if` and `elif`. The URLs have the parameters of our search inserted. 
3. Requests.get fetches the specified webpage. r objects have `.text` attributes which returns the webpage's raw html.
4. BeautifulSoup is a package which parses html and returns a `soup` object.
5. `find_all` takes a html tag as an argument ("div" here). Any argument that’s not recognized (e.g. class_) will be turned into a filter on a tag’s attributes. Here the argument class_, is used to filter against each tag’s 'class_' attribute which identifies a new property.
6. Looping through the apartments, we extract the relevant information this time using `find` and looking for the relevant info indicated by 'class_' again.
7. Appending the info at the end of each loop means we compile all the info across the webpages.

https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=STATION%5E6095&maxPrice=800000&minPrice=400000&radius=3.0&sortType=6&index=24&propertyTypes=detached%2Cflat%2Csemi-detached%2Cterraced&secondaryDisplayPropertyType=housesandflats&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords=

In [4]:
import time

all_apartment_links = [] # stores apartment links
all_description = [] # stores number of bedrooms in the apartment
all_address = [] # stores address of apartment
all_price = [] # stores the listing price of apartment
all_id_no = []

minPrice=450000
maxPrice=800000
radius=3

for i in range(42):
    if i==0:
        r= requests.get(f'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=STATION%5E7658&maxPrice={maxPrice}&minPrice={minPrice}&radius={radius}&sortType=6&propertyTypes=detached%2Cflat%2Csemi-detached%2Cterraced&secondaryDisplayPropertyType=housesandflats&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords=')
    else:
        r = ''
        while r == '':
            try:
                r = requests.get(f'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=STATION%5E7658&maxPrice={maxPrice}&minPrice={minPrice}&radius={radius}&sortType=6&index={i*24}&propertyTypes=detached%2Cflat%2Csemi-detached%2Cterraced&secondaryDisplayPropertyType=housesandflats&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords=')
                break
            except:
                print(f'Connection refused by the server on page {i+1}... sleeping for 3 seconds')
                time.sleep(3)
                print("Was a nice sleep, now let me continue...")
                continue
        #r= requests.get(f'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=STATION%5E6095&maxPrice={maxPrice}&minPrice={minPrice}&radius={radius}&sortType=6&index={i*24}&propertyTypes=detached%2Cflat%2Csemi-detached%2Cterraced&secondaryDisplayPropertyType=housesandflats&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords=')
    
    soup = BeautifulSoup(r.text, 'lxml')
    
    apartments = soup.find_all("div", class_="l-searchResult is-list")
    
    for i in range(len(apartments)):

        # tracks which apartment we are on in the page
        apartment_no = apartments[i]

        # append link
        apartment_info = apartment_no.find("a", class_="propertyCard-link")
        link = "https://www.rightmove.co.uk" + apartment_info.attrs["href"]
        all_apartment_links.append(link)

        # append address
        address = (
            apartment_info.find("address", class_="propertyCard-address")
            .get_text()
            .strip()
        )
        all_address.append(address)

        # append description
        description = (
            apartment_info.find("h2", class_="propertyCard-title")
            .get_text()
            .strip()
        )
        all_description.append(description)

        # append price
        price = (
            apartment_no.find("div", class_="propertyCard-priceValue")
            .get_text()
            .strip()
        )
        all_price.append(price)
        
        # append id
        id_no = (
            apartment_no.find("div", id_="property-*")
            #.get_text()
            #.strip()
        )
        all_id_no.append(id_no)

r.ok

True

In [ ]:
all_links =[]
all_features =[]
all_prices =[]
all_statname =[]
all_statdist =[]
all_outcodes = []
all_postcodes = []
all_centralities = []

import re
import json
import geopy
from geopy import distance

for i in range(len(all_apartment_links)):
    r= requests.get(all_apartment_links[i])

    soup = BeautifulSoup(r.text, 'lxml')
    
    # Append variables of interest
    
    # weblink
    all_links.append(all_apartment_links[i])
    
    # physical features
    features = (
        soup.find_all("p", class_="_1hV1kqpVceE9m-QrX_hWDN")
    )
    features =[str(i).replace('<p class="_1hV1kqpVceE9m-QrX_hWDN">', '').replace('</p>', '')
     .replace('<p class="_1hV1kqpVceE9m-QrX_hWDN _2SpNNVW0fTYoFvPDmhKSt8 _3ixAp8byA3wC3qvii8d-kg">' , '') 
     for i in features]
    features =[i for i in features if "<p class" not in i ]
    all_features.append(features)
    
    # price
    price = (
        soup.find('input').attrs['value']
        #.get_text()
        #.strip()
    )
    all_prices.append(int(price.replace(",","")))
    
    # postcodes   
    links = soup.find_all('script')[4]    
    jsonobj = json.loads(links.text[25:]) # converts json into dictionary  
    outcode = jsonobj.get("propertyData").get('address').get('outcode')
    postcode = outcode + jsonobj.get("propertyData").get('address').get('incode')
    all_outcodes.append(outcode)
    all_postcodes.append(postcode)    
    
    # distance to centre of London
    latitude = float(jsonobj.get("propertyData").get('location').get('latitude'))
    longitude = float(jsonobj.get("propertyData").get('location').get('longitude'))
    coords = (latitude, longitude)
    charingX = (51.507602, -0.127816)
    centrality = geopy.distance.geodesic(coords, charingX).km
    all_centralities.append(centrality)
    
    # stations
    statdist = (
        soup.find("span", class_="_1ZY603T1ryTT3dMgGkM7Lg")
        .get_text()
        .strip()
    )
    all_statdist.append(float(statdist.replace(" miles","")))
    
    statname = (
        soup.find("span", class_="cGDiWU3FlTjqSs-F1LwK4")
        .get_text()
        .strip()
    )
    all_statname.append(statname)

In [ ]:
# Creating property type variable
prop_type =[]

for i in all_features:
    if i[0] in ('Apartment','Flat','Studio','Maisonette', 'House','Terraced','Mews'):
        prop_type.append(i[0])
    else: 
        prop_type.append('')

In [ ]:
# Creating bedrooms & bathrooms variable
bedrooms =[]
bathrooms =[]

import numpy as np

for i in all_features:
    if i[0] == 'Studio':
        bedrooms.append(0)
        if i[1].startswith('×'):
            bathrooms.append(int(i[1][1]))
        else:
            bathrooms.append(np.nan)
    elif i[0].startswith('×') and i[1].startswith('×'): 
        bedrooms.append(int(i[0][1]))
        bathrooms.append(int(i[1][1]))
    elif len(i) <= 2:
        bedrooms.append(np.nan)
        bathrooms.append(np.nan)  
    elif i[1].startswith('×') and i[2].startswith('×'): 
        bedrooms.append(int(i[1][1]))
        bathrooms.append(int(i[2][1]))  
    else:
        bedrooms.append(np.nan)
        bathrooms.append(np.nan)

bedrooms[:20] 
#bathrooms[:20]

In [ ]:
# Creating floor size (sq. ft.) variable

import re
floor_size =[]

for i in range(len(all_features)):
    for j in range(len(all_features[i])):
        if all_features[i][j].endswith('sq. ft.'):
            clean1 = int(re.sub("[^0-9]", "", all_features[i][j]))
            if len(str(clean1)) > 4:   # Fixes instances where floor size is a range (e.g. 439-1051 sq feet)
                clean1 = str(clean1)
                half = int(len(clean1)/2)
                clean1 = int( (int(clean1[0:half]) + int(clean1[half:len(clean1)]) ) / 2)
            elif clean1 > 3000:  # Remove floorsizes > 3000
                clean1 = np.nan
            floor_size.append(clean1)
    if len(floor_size) <= i: 
        floor_size.append(np.nan)
floor_size[:10]

In [ ]:
# Creating contract type variable

contract =[]

for i in all_features:
    if i[-1] == 'Leasehold':
        contract.append('Leasehold')
    elif i[-1] == 'Share of Freehold':
        contract.append('Freehold')
    else:
        contract.append('')

contract[:10]

In [ ]:
# Adding new outcode variable

outcode2 =[]

for i in all_outcodes:
    if i[-1].isalpha() == True:
        outcode2.append(i[0:-1])
    else:
        outcode2.append(i)

In [ ]:
# convert to dataframe
data = {"Links": all_links,
    "Price": all_prices,
    "NearStat": all_statname,
    "StatDist(miles)": all_statdist,
    "Prop_Type": prop_type,
    "Bedrooms": bedrooms,
    "Bathrooms": bathrooms,
    "Floor_Size": floor_size,
    "Contract_Type": contract,
    "Outcode": all_outcodes,
    "Outcode2": outcode2,
    "Postcode": all_postcodes,
    "Centrality": all_centralities
       }
df = pd.DataFrame.from_dict(data)
df.head(10)

In [ ]:
df.loc[df['Floor_Size']>1500]
df.iloc[57]

In [ ]:
len(df)

### Pre-processing

In [ ]:
df['NearStat'].value_counts().head(10)

In [ ]:
df['Outcode2'].value_counts()

In [ ]:
len(df['Outcode'].value_counts()) - len(df['Outcode2'].value_counts())

In [ ]:
# Getting dummies vars
df2 = pd.concat([df, pd.get_dummies(df.iloc[:,4])], axis=1) #prop_type
df2 = pd.concat([df2, pd.get_dummies(df.iloc[:,2])], axis=1) #nearest station
df2 = pd.concat([df2, pd.get_dummies(df.iloc[:,8])], axis=1) #contract
df2 = pd.concat([df2, pd.get_dummies(df.iloc[:,9])], axis=1) #outcode
df2.head(3)

### Variable selection

In [ ]:
# Leave out 'Studio', 'Freehold', 'Regent\'s Park Station', 'Great Portland Street Station', W1W, W1U, W1T, W1G, W1B, W1K
variables = ['Price','StatDist(miles)','Bedrooms','Bathrooms','Apartment','Flat','House','Maisonette','Mews','Terraced','Leasehold','W2', 'NW8', 'NW1', 'W9', 'W1H']
stations = ['Edgware Road (Circle, District, Hammersmith & City) Station',
'St. John\'s Wood Station','Marylebone Station','Paddington Station',                                             
'Edgware Road (Bakerloo) Station', 'Warwick Avenue Station','Lancaster Gate Station',
'Marble Arch Station','Maida Vale Station',
'Baker Street Station','Royal Oak Station']

df_varselect = df2[variables]
df_varselect2 =df2[stations]
df_varselect2 = pd.concat([df_varselect, df_varselect2], axis=1).dropna()
df_varselect2


In [ ]:
from sklearn.linear_model import LinearRegression

X = np.array(df_varselect2.iloc[:,1:df_varselect2.shape[1]])
y = np.array(df_varselect2.iloc[:,0])
reg = LinearRegression().fit(X, y)
reg.score(X, y)

In [ ]:
reg.coef_

### With floor size

In [ ]:
df_varselect3 = df2[variables]
df_varselect4 =df2[stations[0:7]]
df_varselect4 = pd.concat([df_varselect3, df_varselect4, df2['Floor_Size']], axis=1).dropna()
df_varselect4

In [ ]:
len(df_varselect4)

In [ ]:
# R2 is much higher when including floor size
X = np.array(df_varselect4.iloc[:,1:df_varselect4.shape[1]])
y = np.array(df_varselect4.iloc[:,0])
reg = LinearRegression().fit(X, y)
reg.score(X, y)

In [ ]:
from matplotlib import pyplot as plt

x = df_varselect4['Floor_Size']
y = df_varselect4['Price']
a, b, c = np.polyfit(x, y, 2)

plt.figure(figsize=(6, 4))
ax = plt.axes()
ax.scatter(x,y)

x2 = np.linspace(min(x), max(x), len(x))
y2 = a*(x2**2) + b*x2 + c
ax.plot(x2, y2)

ax.set_xlabel('Floor Size (sq. ft.)')
ax.set_ylabel('Price (£)')

plt.show()

In [ ]:
len(x)
